In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
import os

cloned_repo_path = os.path.abspath('')# insert path here
sys.path.insert(0, cloned_repo_path)
cloned_repo_path = os.path.abspath('.')
sys.path.insert(0, cloned_repo_path)

In [ ]:
import os
import glob
import json
import pickle 
import stumpy
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL

## Helping Functions

In [ ]:
def create_directory(directory="Results"):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print("Directory Created Successfully")
    
    else:
        print("Directory Already Exists")

In [ ]:
def calculate_and_visualize_mdls(mps, indices, filepath, m=2016):
    motifs_idx = np.argmin(mps, axis=1)

    # Getting the nearest neighbours of each individual motifs_idx

    nn_idx = indices[np.arange(len(motifs_idx)), motifs_idx]

    mdls, subspaces = stumpy.mdl(final_data, m, motifs_idx, nn_idx)

    print(f"MDLS:\n\t {mdls}\n")

    print(f"Subspaces:\n\t {subspaces}\n")

    k = np.argmin(mdls)

    print(f"Suugested Columns For Multidimensional Matrix Profile:\n\t {final_data.columns[subspaces[k]]}\n")
    
    plt.plot(np.arange(len(mdls)), mdls, c='red', linewidth='4')
    plt.xlabel('k (zero-based)', fontsize='20')
    plt.ylabel('Bit Size', fontsize='20')
    plt.xticks(range(mps.shape[0]))
    
    plt.savefig(filepath, transparent=False, bbox_inches='tight')
    plt.show()
    
    return final_data.columns[subspaces[k]].tolist()

In [ ]:
def mps_calculations_mstump_m(final_data, filepath, m=2016):
    # Calculating Multidimensional Matrix Profile For a weekly rhythm where subsequent length m=2016

    mps, indices = stumpy.mstump_m(final_data, m)

    # Displaying the shape of MultiDimensional Matrix Profile 

    print(f"MPS Shape: {mps.shape}")

    # Getting the motifs_idx based on the standard np.argmin method 
    motifs_idx = np.argmin(mps, axis=1)

    # Displaying the motifs of each dimension
    
    print(f"Motif Start Index: {motifs_idx}")
    

    # Getting the nearest neighbours o
    nn_idx = indices[np.arange(len(motifs_idx)), motifs_idx]
    
    # Displaying the Nearest Neighbors
    
    print(f"Nearest Start Index: {nn_idx}")


    df = final_data.reset_index(drop=True)

    # Plotting the dimensions of the time series 
    fig, axs = plt.subplots(mps.shape[0] * 2, sharex=True, gridspec_kw={'hspace': 0}, figsize=(25, mps.shape[0] * 5))
    label = ''
    for k, dim_name in enumerate(df.columns):
        axs[k].set_ylabel(dim_name, fontsize=10)
        axs[k].set_xlabel('Time', fontsize=10) 
        
        axs[k].plot(df[dim_name], label=sensor_id_type_mapping[dim_name])
 
        axs[k].legend(loc="upper right")
        
        axs[k].plot(range(motifs_idx[k], motifs_idx[k] + m), df[dim_name].iloc[motifs_idx[k] : motifs_idx[k] + m], c='red', linewidth=4)
        axs[k].plot(range(nn_idx[k], nn_idx[k] + m), df[dim_name].iloc[nn_idx[k] : nn_idx[k] + m], c='red', linewidth=4)

        label = label + ', ' + dim_name
        
        axs[k].axvline(x=motifs_idx[k], linestyle="dashed", c='black')
        axs[k].axvline(x=nn_idx[k], linestyle="dashed", c='black')

        axs[k + mps.shape[0]].set_ylabel(f"P_{k}", fontsize=10)
        axs[k + mps.shape[0]].plot(mps[k], c='orange', label=f"{label.strip(',')}")
        axs[k + mps.shape[0]].set_xlabel('Time', fontsize=10)    

        axs[k + mps.shape[0]].axvline(x=motifs_idx[k], linestyle="dashed", c='black')
        axs[k + mps.shape[0]].axvline(x=nn_idx[k], linestyle="dashed", c='black')    

        axs[k + mps.shape[0]].plot(motifs_idx[k], mps[k, motifs_idx[k]] + 1, marker="v", markersize=10, color='red')
        axs[k + mps.shape[0]].plot(nn_idx[k], mps[k, nn_idx[k]] + 1, marker="v", markersize=10, color='red')
        
        axs[k + mps.shape[0]].text(motifs_idx[k], mps[k, motifs_idx[k]], f"{1}m", fontsize="xx-large")
        axs[k + mps.shape[0]].text(nn_idx[k], mps[k, motifs_idx[k]], f"{1}n", fontsize="xx-large")
        axs[k + mps.shape[0]].legend(loc="upper right")
        
    plt.savefig(filepath, transparent=False, bbox_inches='tight')
    plt.show()
    
    return mps, indices



In [ ]:
def calculate_nn_for_prev_dimensions(indices, motifs_idx):
    nn_idx = indices[np.arange(len(motifs_idx)), motifs_idx]
    
    return nn_idx

In [ ]:
def mps_calculates_nn(final_data, mps, indices, filepath, m=2016):
    
    # Getting the motifs_idx based on the standard np.argmin method 
    motifs_idx = np.argmin(mps, axis=1)

    # Resetting the date index 

    df = final_data.reset_index(drop=True)

    # Plotting the dimensions of the time series 
    
    for i in range(len(df.columns)):
        label = ''
        fig, axs = plt.subplots(i + 2, sharex=True, gridspec_kw={'hspace': 0}, figsize=(25, (i + 1) * 5))
        sub_motifs_idx = [motifs_idx[i]] * (i+1) 
        nn_idx = calculate_nn_for_prev_dimensions(indices, sub_motifs_idx)
        print(sub_motifs_idx)
        print(nn_idx)
        
        for k, dim_name in enumerate(df.columns[:i+1]):

            axs[k].set_ylabel(dim_name, fontsize=10)
            axs[k].set_xlabel('Time', fontsize=10) 

            axs[k].plot(df[dim_name], label=sensor_id_type_mapping[dim_name])

            axs[k].legend(loc="upper right")

            axs[k].plot(range(sub_motifs_idx[k], sub_motifs_idx[k] + m), df[dim_name].iloc[sub_motifs_idx[k] : sub_motifs_idx[k] + m], c='red', linewidth=4)
            axs[k].plot(range(nn_idx[k], nn_idx[k] + m), df[dim_name].iloc[nn_idx[k] : nn_idx[k] + m], c='red', linewidth=4)

            label = label + ', ' + dim_name
            axs[k].axvline(x=sub_motifs_idx[k], linestyle="dashed", c='black')
            axs[k].axvline(x=nn_idx[k], linestyle="dashed", c='black')

        new_mps = mps[:i+1] 

        axs[i + 1].set_ylabel(f"P_{i}", fontsize=10)
        axs[i + 1].plot(new_mps[-1], c='orange', label=f"{label.strip(',')}")
        axs[i + 1].set_xlabel('Time', fontsize=10)    

        axs[i + 1].axvline(x=motifs_idx[i], linestyle="dashed", c='black')
        axs[i + 1].plot(motifs_idx[i], mps[i, motifs_idx[i]] + 1, marker="v", markersize=10, color='red')
        axs[i + 1].text(motifs_idx[i], mps[i, motifs_idx[i]], f"{1}m", fontsize="xx-large")
        axs[i + 1].legend(loc="upper right")
        
        for j in range(i+1):
            axs[i + 1].axvline(x=nn_idx[j], linestyle="dashed", c='black')
            axs[i + 1].plot(nn_idx[j], mps[i, nn_idx[j]] + 1, marker="v", markersize=10, color='red')
            axs[i + 1].text(nn_idx[j], mps[i, nn_idx[j]], f"{j+1}n", fontsize="xx-large")
       
        plt.savefig(f"{filepath}_{i+1}.jpg", transparent=False, bbox_inches='tight')
        plt.show()



In [ ]:
# Further Processing 
def get_percent_motif_start_index(motifs):
    final_motifs = []
    
    for i in range(0, len(motifs), 2016):
        final_motifs.append(motifs.index[i])
            
    return final_motifs

In [ ]:
# Further Processing the motifs to get the first index
def get_motif_start_index(motifs):
    final_motifs = []
    j = 0
    check = False
    
    for i in range(0, len(motifs), 2016):
        if not check:
            j = i

        final_motifs.append(motifs[j])

        if check:
            j = i + value

        elif motifs[i] + 2016 > len(motifs):
            value = len(motifs) - motifs[i]
            j = i + value
            check = True
            
                
    return final_motifs


In [ ]:
# A function that would take in percentage for discord and percentage for motif 
def select_motifs_discords_percentage(mps, dimension, motif_percentage, discord_percentage, motifs={}, discords={}):
    motif_threshold = mps.quantile(motif_percentage/100)
    discord_threshold = mps.quantile((100 - discord_percentage)/100)
    
    motif = mps[mps < motif_threshold]
    discord = mps[mps > discord_threshold]
    
    if len(motif):
        
        motifs[dimension] = get_percent_motif_start_index(motif)
        
    return motifs, discords

In [ ]:
# Select the Upper K of all points and Lowest J of all points as discords and motifs from mps

def select_top_k_motifs_discords(mps, dimension, k_motifs, k_discords, motifs={}, discords={}):
    # Getting the motifs_idx based on the standard np.argmin method 
    # It gives 1 single motif for each dimension
    sorted_mps = np.argsort(mps, kind='stable')
    
    motifs[dimension] = get_motif_start_index(sorted_mps)[:k_motifs]
    
    return motifs, discords

In [ ]:
# Taking user's input for function to be applied
motif_thresholds_for_all_dimensions = {}

def take_function_and_function_parameters_input():
    motif_thresholds_for_all_dimensions = {}
    for dimension in range(mps.shape[0]):
        motif_thresholds_for_single_dimension = {}
        function = input(f"Select A Function To Be Applied to Dimension {dimension}\n"
                         f"\tPress 1 For Selection Based on Percentage\n "
                         f"\tPress 2 For Top K Motifs and Discords Selection: ")

        if int(function) == 1:
            motif_percentage = input(f"\tEnter a Specific Threshold Value For Motif Selection For Dimension {dimension}: ")
            discord_percentage = input(f"\tEnter a Specific Threshold Value For Discord Selection For Dimension {dimension}: ")
            motif_thresholds_for_single_dimension["function"] = int(function)
            motif_thresholds_for_single_dimension["motif_percentage"] = int(motif_percentage)
            motif_thresholds_for_single_dimension["discord_percentage"] = int(discord_percentage)

        elif int(function) == 2:
            k_motif = input(f"\tEnter Top K Motif Selection For Dimension {dimension}: ")
            k_discord = input(f"\tEnter Top K Discord Selection For Dimension {dimension}: ")
            motif_thresholds_for_single_dimension["function"] = int(function)
            motif_thresholds_for_single_dimension["k_motifs"] = int(k_motif)
            motif_thresholds_for_single_dimension["k_discords"] = int(k_discord)
        else:
            continue
        motif_thresholds_for_all_dimensions[dimension] = motif_thresholds_for_single_dimension
        
    return motif_thresholds_for_all_dimensions


In [ ]:
# Calculating Motifs and Discords 
def calculate_motifs_discords_for_each_dimension(mps, motif_thresholds_for_all_dimensions):
    motifs = {}
    discords = {}

    mps_df = pd.DataFrame(mps).T

    for key, value in motif_thresholds_for_all_dimensions.items():
        if value.get('function') == 1:
            motifs, discords = select_motifs_discords_percentage(mps_df[key], key, value["motif_percentage"],
                                                                 value["discord_percentage"], motifs, discords)
        elif value.get('function') == 2:
            motifs, discords = select_top_k_motifs_discords(mps_df[key], key, value['k_motifs'], 
                                                            value['k_discords'], motifs, discords)
    return motifs, discords

In [ ]:
# Selecting Nearest Neighbour that correspond to each motif_idx in the final_motifs dict

def calculate_nn_and_filter_motifs(motifs):
    final_nn = {}
    final_motifs = {}

    for key, values in motifs.items():
        nns = {}
        for value in values:
            if nns.get(value) is None:
                if final_nn.get(key):
                    final_nn[key].append(indices[key, value])
                    final_motifs[key].append(value)
                else:
                    final_nn.setdefault(key, []).append(indices[key, value])
                    final_motifs.setdefault(key, []).append(value)

                nns[indices[key, value]] = 1
    
    return final_motifs, final_nn

In [ ]:

def create_sensor_id_type_mapping(location_input):
    sensors_id_type = sensor_location_type_ids.loc[:,location_input].to_dict()

    sensor_id_type_mapping = {}
    for key, values in sensors_id_type.items():
        if isinstance(values, str)  and 'list' not in key:
            for value in eval(values.replace(' ', ',')):
                sensor_id_type_mapping[f"{value}"] = key
            
    return sensor_id_type_mapping

In [ ]:
# Plotting the dimensions of the time series dataframe df along with the dimensions of the matrix profile

def plot_results_of_mps(df, mps, m, final_motifs, final_nn, sensor_id_type_mapping, filepath):
    fig, axs = plt.subplots(mps.shape[0] * 2, sharex=True, gridspec_kw={'hspace': 0}, figsize=(25, mps.shape[0] * 5))
    label = ''
    for k, dim_name in enumerate(df.columns):
        axs[k].set_ylabel(dim_name, fontsize=10)
        axs[k].set_xlabel('Time', fontsize=10) 
        axs[k].plot(df[dim_name], label=sensor_id_type_mapping[dim_name])
        axs[k].legend(loc="upper right")
        i = 0
        if final_motifs.get(k) and final_nn.get(k):
            for motifs_idx, nn_idx in zip(final_motifs.get(k), final_nn.get(k)):
                
                axs[k].plot(df[dim_name].iloc[motifs_idx : motifs_idx + m], c='red', linewidth=4)
                axs[k].plot(df[dim_name].iloc[nn_idx : nn_idx + m], c='red', linewidth=4)
                axs[k].axvline(x=motifs_idx, linestyle="dashed", c='black')
                axs[k].axvline(x=nn_idx, linestyle="dashed", c='black')

                axs[k + mps.shape[0]].plot(motifs_idx, mps[k, motifs_idx] + 1, marker="v", markersize=10, color='red')
                axs[k + mps.shape[0]].plot(nn_idx, mps[k, nn_idx] + 1, marker="v", markersize=10, color='red')

                axs[k + mps.shape[0]].axvline(x=motifs_idx, linestyle="dashed", c='black')
                axs[k + mps.shape[0]].axvline(x=nn_idx, linestyle="dashed", c='black')

                axs[k + mps.shape[0]].text(motifs_idx, mps[k][motifs_idx], f"{i+1}m", fontsize="xx-large")
                axs[k + mps.shape[0]].text(nn_idx, mps[k][nn_idx], f"{i+1}n", fontsize="xx-large")

                i += 1
        label = label + ', ' + dim_name
        
        axs[k + mps.shape[0]].set_ylabel(f"P_{k}", fontsize=10)
        axs[k + mps.shape[0]].plot(mps[k], c='orange', label=f"{label.strip(',')}")
        axs[k + mps.shape[0]].set_xlabel('Time', fontsize=10)
        axs[k + mps.shape[0]].legend(loc="upper right")

    plt.savefig(filepath, transparent=False, bbox_inches='tight')
    plt.show()


In [ ]:
def apply_functions(final_data, mps, m, filepath):
    motif_thresholds_for_all_dimensions = take_function_and_function_parameters_input()
        print(f"\nMotif Threshold Dictionary:\n\t {motif_thresholds_for_all_dimensions}\n")

    motifs, discords = calculate_motifs_discords_for_each_dimension(mps, motif_thresholds_for_all_dimensions)
    print(f"Motifs Before Filtering:\n\t {motifs}\n")

    final_motifs, final_nn = calculate_nn_and_filter_motifs(motifs)

    # Resetting the date index 
    df = final_data.reset_index(drop=True)

    sensor_id_type_mapping = create_sensor_id_type_mapping(location_input)

    plot_results_of_mps(df, mps, m, final_motifs, final_nn, sensor_id_type_mapping, filepath)
    
    return final_motifs, final_nn

In [ ]:
def apply_functions_heatmap(final_data, mps, m, filepath, 
                            motif_thresholds_for_all_dimensions):
    
    print(f"\nMotif Threshold Dictionary:\n\t {motif_thresholds_for_all_dimensions}\n")

    motifs, discords = calculate_motifs_discords_for_each_dimension(mps, motif_thresholds_for_all_dimensions)
    print(f"Motifs Before Filtering:\n\t {motifs}\n")

    final_motifs, final_nn = calculate_nn_and_filter_motifs(motifs)

    # Resetting the date index 
    df = final_data.reset_index(drop=True)

    sensor_id_type_mapping = create_sensor_id_type_mapping(location_input)

    plot_results_of_mps(df, mps, m, final_motifs, final_nn, sensor_id_type_mapping, filepath)
    
    return final_motifs, final_nn

In [ ]:
def select_top_k_motifs_heatmap(final_data, mps, m, filepath, json_path, 
                                motif_thresholds_for_all_dimensions):
    
    final_motifs, final_nn = apply_functions_heatmap(final_data, mps, 2016, filepath, 
                                                    motif_thresholds_for_all_dimensions)

    # Creating a list of dictionaries that will be dumped to json file
    # Each dictionary in the list will have 2 keys, 
    json_data = []

    for k in range(final_data.shape[1]):
        values = {'k': k}
        i = 1
        subspaces = []
        
        if final_motifs and final_motifs.get(k):
            
            for motifs_idx, nn_idx in zip(final_motifs[k], final_nn[k]):
                subspace = stumpy.subspace(final_data, 2016, motifs_idx, nn_idx, k)
                motif_subspaces = {"motif_id": i, "motif_idx": int(motifs_idx), "nn_idx": int(nn_idx), "subspace_column": subspace.tolist(), "subspace_sensor_ids": final_data.columns[subspace].tolist()}
                i += 1
                
                print(f"For k = {k}, the {k + 1}-dimensional subspace includes subsequences from {subspace}")
                subspaces.append(motif_subspaces)
        
        values['motif_subspaces'] = subspaces
        json_data.append(values)

    # Dumping the motifs and nearest neighbors in a specific dimension 

    with open(json_path, 'w') as f:
        json.dump(json_data, f, indent=2)

## Creating Folders and Reading Preprocessed Data Files

In [ ]:
create_directory(directory="Results_Subspaces")

In [ ]:
sensor_location_type_ids = pd.read_csv('./Processed_Data/Sensor_Location_Type_Ids.csv', index_col='name')

In [ ]:
with open('./Processed_Data/sensor_type_names_dict.pkl', 'rb') as f:
    sensor_type_names_dict = pickle.load(f)

In [ ]:
location_input = input("Enter a Specific Location For MMP: ")

In [ ]:
directory = "Results_Subspaces/" + location_input.replace('/', '_')
create_directory(directory)

In [ ]:
# directory1 = directory + '/All_Dimenisons_Subspaces'
# create_directory(directory1)

In [ ]:
# Getting list of sensor_ids/column

columns = []
for key in sensor_type_names_dict.keys():
    columns.extend(eval(sensor_location_type_ids.loc[f'{key}_list', location_input]))
    
columns.append('Timestamp')

In [ ]:
final_data = pd.read_csv("./Processed_Data/Final_Sensor_Time_Series_Imputed.csv", usecols=columns, 
                         index_col='Timestamp', parse_dates=True)


In [ ]:
# Displaying Time Series DataFrame 
final_data

In [ ]:
final_data.to_csv(f"{directory}/df.csv")

In [ ]:
# Creating a Mapping
sensors_id_type = sensor_location_type_ids.loc[:,location_input].to_dict()

sensor_id_type_mapping = {}
for key, values in sensors_id_type.items():
    if isinstance(values, str)  and 'list' not in key:
        for value in eval(values.replace(' ', ',')):
            sensor_id_type_mapping[f"{value}"] = key

In [ ]:
sensor_id_type_mapping

### MPS Calculation Using Constraint Algorithm For All Dimensions (Weekly Rhythm)

In [ ]:
mps, indices = mps_calculations_mstump_m(final_data, 
                                         f"{directory}/All_Dimensions_mstump_m_Weekly.jpg", 
                                         m=2016)

### MPS Calculation Using Constraint Algorithm For All Dimensions With Top Motifs (Weekly Rhythm)

In [ ]:
threshold_input = input("How Many Times Do you want to select Top K Motifs (Build Json Data): ")

In [ ]:
for i in range(1, int(threshold_input)+1):
    motif_thresholds_for_all_dimensions = {}
    motif_thresholds_for_single_dimension = {}
    for dimension in range(final_data.shape[1]):
        motif_thresholds_for_single_dimension["function"] = 2
        motif_thresholds_for_single_dimension["k_motifs"] = i
        motif_thresholds_for_single_dimension["k_discords"] = i
        motif_thresholds_for_all_dimensions[dimension] = motif_thresholds_for_single_dimension
    
    select_top_k_motifs_heatmap(final_data, mps, 2016, 
                                f"{directory}/All_Dimensions_mstump_m_Weekly_Top_{i}_motifs.jpg", 
                                f"{directory}/Subspaces_Data_Weekly_Top_{i}_motifs.json",
                               motif_thresholds_for_all_dimensions)

In [ ]:
# for values in json_data:
#     directory2 = f"{directory1}/k={values['k']}"
#     create_directory(f"{directory2}")
#     for motif in values["motif_subspaces"]:
#         mps, indices = mps_calculations_mstump_m(final_data[motif["subspace_sensor_ids"]], 
#                                          f"{directory2}/Motif_Id_{motif['motif_id']}.jpg", 
#                                          m=2016)

### Calculating the Change in the Subspaces and Building a Heatmap 

In [ ]:
def count_changed_subspaces(motif_subspaces, sort=False):
    change_count = 0
    if not motif_subspaces:
        return None
    
    dataframe = []
    
    for subspace in motif_subspaces:
        if sort:
            dataframe.append({"subspaces": "".join(sorted(subspace["subspace_sensor_ids"]))})
        else:
            dataframe.append({"subspaces": "".join(subspace["subspace_sensor_ids"])})
    
    dataframe = pd.DataFrame(dataframe)
    
    return dataframe["subspaces"].nunique()

In [ ]:
files = sorted(glob.glob(f"{directory}/*.json"))

In [ ]:
files = sorted(glob.glob(f"{directory}/*.json"))
motif_count = 0
dimensions_change_count_motifs = []

for file in files:
    motif_count += 1
    with open(file, 'r') as f:
        json_data = json.load(f)

    dimensions_change_count = {}
    highest = 0

    for value in json_data:
        dimensions_change_count[value['k']] = {"Total_Motifs": motif_count, 
                                               "Changed_Count": count_changed_subspaces(value["motif_subspaces"])}
    
    dimensions_change_count_motifs.append(dimensions_change_count)    
    

In [ ]:
dimensions_change_count_motifs

In [ ]:
matrix = np.nan * np.ones(shape=(len(dimensions_change_count_motifs), len(dimensions_change_count_motifs[0])))

In [ ]:
for i in range(len(dimensions_change_count_motifs)):
    for key, value in dimensions_change_count_motifs[i].items():
        matrix[i][key] = value["Changed_Count"]/value["Total_Motifs"]

In [ ]:
df = pd.DataFrame(matrix)

In [ ]:
df

In [ ]:
fig, ax = plt.subplots(figsize=(len(dimensions_change_count_motifs[0]), len(dimensions_change_count_motifs)))
hm = sns.heatmap(data = df, annot=True, linewidths=2, ax=ax)
hm.set(xlabel='k', ylabel='Motifs_Count')
ax.invert_yaxis()
plt.savefig(f"{directory}/heatmap.jpg", transparent=False, bbox_inches='tight')
plt.show()